# Counting miniflash crashes

This script processes the output from the partition_data notebook in order to count the number of miniflash crashes and the number of equities having miniflash crashes each day. The output of this script is in the data/ directory of this repository. The script also counts the number of trades in the [09:30,09:33) morning window each day.

In [1]:
from datetime import datetime, date
from datetime import timedelta
import math
import glob
import random

DATES=["20141001","20141002","20141003","20141006","20141007","20141008","20141009","20141010","20141013","20141014","20141015"]

LETTERS="abcdefghijklmnopqrstuvwxyz"
sym_table={}

def make_unq_sym(sym):
    if not sym in sym_table:
        while True:
            code=[LETTERS[random.randint(0,25)] for i in range(0,4)] #Four random letters
            code="".join(code) #Joined together as one string
            if not code in sym_table.values():
                sym_table[sym]=code
                break
    return sym_table[sym]

# DEFINITION OF A DOWN (UP) CRASH:
#  1) Has to tick down (up) at least ten times before ticking up (down)
#  2) Price changes have to occur within 1.5 seconds
#  3) Price changes have to exceed -0.8% (+0.8%)

#Set the thresholds
TICK_THRESHOLD = 10
TIME_THRESHOLD = timedelta(seconds=1, microseconds=500000)
PRICE_THRESHOLD = 0.008


def checkForCrash(tradeList):
        if len(tradeList)>TICK_THRESHOLD: #NB Must be strictly greater than (a list of 5 trades has four declines)
            timeDelta = tradeList[len(tradeList)-1]["time"] - tradeList[1]["time"]
            priceStart = tradeList[0]["price"]
            priceDelta = (tradeList[len(tradeList)-1]["price"] - priceStart)/priceStart
            if timeDelta<TIME_THRESHOLD and math.fabs(priceDelta)>=PRICE_THRESHOLD:
                output={
                    "symbol": make_unq_sym(tradeList[0]["ticker"]),
                    "start_time":tradeList[1]["time"],
                    "start_price":tradeList[0]["price"],
                    "end_time":tradeList[len(tradeList)-1]["time"],
                    "end_price":tradeList[len(tradeList)-1]["price"],
                    "price_change":priceDelta,
                    "ticks":len(tradeList)
                }
                return output
        return None


#NB: We rely on the fact that the trades are in the correct temporal order.

def processFile(file):
    previousPrice=None
    tradeList=[]
    direction=0 #What direction have we been moving? -1 for down, +1 for up, 0 for flat
    miniflash_crashes=[]
    
    for line in file:
        tradeAttributes = line.split("\t")

        trade={
            "time": datetime.strptime("{} {}".format(tradeAttributes[0],tradeAttributes[2]),"%Y%m%d %H:%M:%S.%f"),
            "ticker": tradeAttributes[1][1:],
            "price": float(tradeAttributes[4])
        }

        #First time only: set the opening price and add the trade
        if previousPrice==None:
            previousPrice=trade["price"]
            tradeList=[trade]
            continue

        #if trade["time"]<previousTime:
        #	ERROR. This should never happen.

        if trade["price"]<previousPrice:
            if direction==1: #Reset list, we were previously moving up, but now we are going down
                l=checkForCrash(tradeList)
                if l!=None:
                    miniflash_crashes.append(l)
                tradeList=[tradeList[len(tradeList)-1]]
            direction=-1
            tradeList.append(trade)
        elif trade["price"]>previousPrice:
            if direction==-1: #Reset list
                l=checkForCrash(tradeList)
                if l!=None:
                    miniflash_crashes.append(l)
                tradeList=[tradeList[len(tradeList)-1]]
            direction=1
            tradeList.append(trade)

        #Regardless, set the previous price
        previousPrice=trade["price"]
    return miniflash_crashes





In [2]:
#Find all mini-flash crashes (MFCs) and write these to files (one per day)
for DATE in DATES:
    print(DATE)
    
    total_trades=0
    
    #Set output file
    fhMFC = open("../data/{}_miniflash_crashes.csv".format(DATE), "w")
    fhMFC.write("symbol,start_time,end_time,price_change,ticks\n")

    for file in glob.glob("output/output_{}/*".format(DATE)):
        #print(file)
        with open(file,"r") as f:
            mfcs=processFile(f)
        for mfc in mfcs:
            output="{symbol},{start_time},{end_time},{price_change},{ticks}\n".format(**mfc)
            fhMFC.write(output)            
    fhMFC.close()

print("All done.")

20141001
20141002
20141003
20141006
20141007
20141008
20141009
20141010
20141013
20141014
20141015
All done.


In [3]:
#Trades in 09:30 to 09:33 window each day

fhVolume=open("../data/trade_volume.csv","w")
fhVolume.write("date\ttrades_09:30-09:33\n")


START_TIME=datetime.strptime("09:30:00.0","%H:%M:%S.%f").time() #Greater than or equal to this
END_TIME=datetime.strptime("09:33:00.0","%H:%M:%S.%f").time() #Strictly less than this

print("Date\tTrades between 09:30 and 09:33")
for date in DATES:
    count=0
    for file in glob.glob("output/output_{}/*".format(date)):
        with open(file,"r") as fh:
            for line in fh:
                tradeAttributes = line.split("\t")
                dateCalc = datetime.strptime(tradeAttributes[2], "%H:%M:%S.%f").time()
                if dateCalc >= START_TIME and dateCalc < END_TIME:
                    count += 1
                if dateCalc > END_TIME:
                    break
    print("{}\t{}".format(date, count))
    fhVolume.write("{}\t{}\n".format(date, count))
fhVolume.close()

Date	Trades between 09:30 and 09:33
20141001	354394
20141002	357221
20141003	364482
20141006	377942
20141007	368060
20141008	320907
20141009	354617
20141010	593973
20141013	376676
20141014	444223
20141015	730379
